# Notebook dedicated to the github parsing for our RIMEL project
## Authors : 
- Fabien Airaud
- Axel Delille
- Yvann Ponce
- Apoorva Srinivas Appadoo
---

In [1]:
%pip install selenium

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.7 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.7 MB 3.2 MB/s eta 0:00:03
   ----------- ---------------------------- 2.9/9.7 MB 4.2 MB/s eta 0:00:02
   ----------------- ---------------------- 4.2/9.7 MB 4.7 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.7 MB 4.7 MB/s eta 0:00:01
   ------------------------- -------------- 6.3/9.7 MB 4.7 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.7 MB 4.7 MB/s eta 0:00:01
   --------------------------------- ------ 8.1/9.7 MB 4.8 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.7 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 4.8 MB/s eta 0:00:00
Note: you may need to re


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from dotenv import load_dotenv
from os import getenv

import urllib.parse
import json


First step is to prepare all variables required

In [3]:
def load_and_process_config(file_path):
    with open(file_path, 'r') as file:
        config_data = json.load(file)
    return config_data

In [8]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")

# Fetching github credentials
load_dotenv()
githubUsername = getenv("GH_USERNAME")
githubPassword = getenv("GH_PASSWORD")

# Config filepath, relative to this file
config = "conf.json"
output_file = "output.json"

Here we define the utilities method to parse the github search page

In [5]:
def convert_github_placeholder_to_int(value_str):
    # Remove any extra spaces
    value_str = value_str.strip()

    # Handle different suffixes
    if value_str.endswith('k'):
        return int(float(value_str[:-1]) * 1000)
    elif value_str.endswith('M'):
        return int(float(value_str[:-1]) * 1000000)
    elif value_str.endswith('B'):
        return int(float(value_str[:-1]) * 1000000000)
    else:
        # If no suffix, simply convert the string to an integer
        return int(value_str)

def parseGithubSearch(driver: webdriver, url: str) -> int :
    driver.get(url)

    # Wait for the page to be fully loaded
    WebDriverWait(driver, 10).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )
    
    # Step 6: Perform any further actions (for example, scraping the page)
    # Locate the div with class "Box-sc-g0xbh4-0 cAMcRf" and find the span inside it
    span_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH, "//div[contains(@class, 'Box-sc-g0xbh4-0') and contains(@class, 'cAMcRf')]//span[contains(@class, 'prc-Text-Text-0ima0')]"
        ))
    )

    # Extract the text from the span
    span_text = span_element.text
    return convert_github_placeholder_to_int(span_text)

def generateGithubParsingFromParams(driver, codeToSearch: str, extensions: list[str]) -> int :
    extensionString : str = ""
    
    if not extensions or extensions == [] :
        extensionString = ""  # Return empty string if the list is empty
    
    # If there's only one extension, don't add 'OR'
    if len(extensions) == 1:
        extensionString = f"path:*.{extensions[0]}"
    
    if len(extensions) > 1 :
        extensionString = f'({" OR ".join([f"path:*.{ext}" for ext in extensions])})'
    
    query : str = f'"{codeToSearch}" {extensionString}'
    finalString = f'https://github.com/search?q={urllib.parse.quote(query)}&type=code'
    return parseGithubSearch(driver, finalString)

def process_config_and_fetch_numbers(driver, file_path):
    config = load_and_process_config(file_path)

    for item in config:
        codeToSearch = item['query']
        extensions = item['extensions']
        number_of_results = generateGithubParsingFromParams(driver, codeToSearch, extensions)
        item["number"] = number_of_results
    
    return config

Now that everything is set up we'll do the parsing for real

In [6]:
# Step 1: Go to GitHub login page
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://github.com/login")

# Step 2: Wait for the username and password fields to be visible and fill them
username_field = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "login_field"))
)
password_field = driver.find_element(By.ID, "password")

# Fill in the login credentials
username_field.send_keys(githubUsername)
password_field.send_keys(githubPassword)

# Submit the form by clicking the "Sign in" button
login_button = driver.find_element(By.NAME, "commit")
login_button.click()

# Step 3: Wait for the login to complete (you can adjust this based on what you want to check)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//span[contains(text(),'Pull requests')]"))
)

# Step 4: Now you're logged in, you can go to your search page or other pages
results = process_config_and_fetch_numbers(driver, config)

# Step 7: Close the browser
driver.quit()

Save the result into a json to not rerun the parsing to see the data vizualisation again

In [7]:
print(results)

[{'tool': 'cml', 'query': 'iterative/setup-cml@v1', 'extensions': ['yml', 'yaml'], 'number': 716}, {'tool': 'cml', 'query': 'cml runner launch', 'extensions': ['yml', 'yaml', 'sh'], 'number': 50}, {'tool': 'cml', 'query': 'cml comment create', 'extensions': ['yml', 'yaml', 'sh'], 'number': 243}, {'tool': 'cml', 'query': 'cml check create', 'extensions': ['yml', 'yaml', 'sh'], 'number': 6}, {'tool': 'cml', 'query': 'cml pr create', 'extensions': ['yml', 'yaml', 'sh'], 'number': 26}, {'tool': 'cml', 'query': 'cml tensorboard connect', 'extensions': ['yml', 'yaml', 'sh'], 'number': 3}, {'tool': 'ClearML', 'query': 'from clearml.automation', 'extensions': ['py', 'ipynb'], 'number': 271}, {'tool': 'ClearML', 'query': 'from clearml import Task', 'extensions': ['py', 'ipynb'], 'number': 1900}, {'tool': 'ClearML', 'query': 'clearml-agent', 'extensions': ['py', 'ipynb'], 'number': 302}, {'tool': 'ClearML', 'query': 'clearml-session', 'extensions': ['py', 'ipynb'], 'number': 5}, {'tool': 'ClearM

In [10]:
with open(output_file, 'w') as json_file:
    json.dump(results, json_file, indent=4)

print(f"Data saved to {output_file}")

Data saved to output.json
